In [1]:
from ethomap import DynamicTimeWarping
import numpy as np

In [3]:
# Create some 3D time series
x = np.random.rand(200, 3)
y = np.random.rand(200, 3)

# Align time series y to x
dtw = DynamicTimeWarping(x)
dist = dtw.align(y)
print("DTW distance between x and y:", dist)

DTW distance between x and y: 109.34883211924036


In [4]:
from ethomap import pdist_dtw, affinity_propagation, IsomapPrecomputed
import numpy as np
import time
from scipy.spatial.distance import squareform

In [6]:
# Create some 3D time series to align pairwise
xs = np.random.rand(1000, 50, 3)
xs

array([[[0.18662518, 0.10489391, 0.52162114],
        [0.67142212, 0.25820038, 0.85673517],
        [0.03119912, 0.08938447, 0.62547084],
        ...,
        [0.0993939 , 0.80890089, 0.95989614],
        [0.32842231, 0.57552049, 0.37370693],
        [0.08086257, 0.22550851, 0.31769072]],

       [[0.79610176, 0.51613982, 0.10898017],
        [0.19276936, 0.29641703, 0.30571956],
        [0.37110069, 0.3766464 , 0.54726069],
        ...,
        [0.07684084, 0.60030969, 0.93683701],
        [0.70704435, 0.35411822, 0.35981255],
        [0.61860726, 0.34006693, 0.57221085]],

       [[0.62078509, 0.00560585, 0.61421453],
        [0.2741044 , 0.85594989, 0.98283239],
        [0.49624098, 0.20458488, 0.64859466],
        ...,
        [0.46748135, 0.64244565, 0.74545272],
        [0.5926747 , 0.57830149, 0.01689694],
        [0.74959983, 0.11598172, 0.31388013]],

       ...,

       [[0.4224076 , 0.30467123, 0.8050036 ],
        [0.27802096, 0.20095142, 0.98606455],
        [0.5461115 , 0

In [8]:
# Compute pairwise distances between time series (pdist_dtw from ethomap.distance)
t0 = time.time()
distances = pdist_dtw(xs, parallel_processing=True, n_processors=4, bw=0.1, fs=100)
distance_matrix = squareform(distances)
t1 = time.time()
print("Time taken:", t1 - t0, "seconds") # ~30 seconds for data of this size
print(distance_matrix.shape) # (1000,1000)

Time taken: 30.850045919418335 seconds
(1000, 1000)


In [9]:
# Create behavioural space from all data(IsomapPrecomputed from ethomap.embed)
isomap = IsomapPrecomputed(n_neighbors=5, n_components=2)
embedding = isomap.fit_transform(distance_matrix)
print(embedding.shape) # (1000,2)

(1000, 2)


In [11]:
# Find exemplars by clustering distance matrix (affinity_propagation from ethomap.cluster)
ap = affinity_propagation(distance_matrix, preference="median")
cluster_labels = ap.labels_
exemplar_indices = ap.cluster_centers_indices_

In [12]:
# Select exemplars from original data
exemplars = xs[exemplar_indices]
exemplar_distances = distance_matrix[exemplar_indices, :][:, exemplar_indices]
print(exemplars.shape) # (n_exemplars, 50, 3)
print(exemplar_distances.shape) #(n_exemplars, n_exemplars)

(48, 50, 3)
(48, 48)


In [13]:
# Create behavioural space from exemplars only
embedding_exemplars = isomap.fit_transform(exemplar_distances)
print(embedding_exemplars.shape)

(48, 2)
